# IFN509 Assignment 2

In [5]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np
import os

In [8]:
# Detect environment
IN_COLAB = 'COLAB_GPU' in os.environ or 'google.colab' in str(get_ipython())

# Define data path logic
if IN_COLAB:
    # Option 1: Load directly from GitHub (recommended for read-only data)
    url = "https://github.com/edanfisher/QUT/blob/main/data/Dataset.csv"
    df = pd.read_csv(url)

    # Option 2: (if data is large/private)
    # from google.colab import drive
    # drive.mount('/content/drive')
    # df = pd.read_csv('/content/drive/MyDrive/path/to/data.csv')
else:
    # Local path for your machine
    df = pd.read_csv("data/Dataset.csv")

HTTPError: HTTP Error 404: Not Found

## Task 1 - Examining Data Types

In [ ]:
df.head()

We'll use .info() to examine the datatypes and .isnull() to examine null values - we'll process these later

In [ ]:
df.info()

In [ ]:
df.isnull().sum().sort_values(ascending=False)

# Column Data Types & Missing Data Review

| #  | Column                  | Non-Null Count | Current Dtype       | Suggested Dtype         | Missing Values / Notes |
|----|-------------------------|----------------|-------------------|------------------------|-----------------------|
| 0  | Participant_ID          | 5789           | int64             | int64                  | 0, fine |
| 1  | survey_date             | 4874           | datetime64[ns]    | datetime64[ns]         | 915 missing; may have malformed dates |
| 2  | region                  | 5774           | category          | category               | 15 missing |
| 3  | country                 | 5786           | category          | category               | 3 missing |
| 4  | ip_latitude             | 5789           | float64           | float64                | 0 missing, fine |
| 5  | ip_longitude            | 5789           | float64           | float64                | 0 missing, fine |
| 6  | gender                  | 5789           | category          | category               | 0 missing, fine |
| 7  | age                     | 5783           | category          | ordered category       | 6 missing; stored as ranges |
| 8  | height                  | 5789           | int64             | int64                  | 0 missing, fine |
| 9  | weight                  | 5789           | int64             | int64                  | 0 missing, fine |
| 10 | bmi                     | 5789           | float64           | float64                | 0 missing, fine |
| 11 | blood_type              | 5689           | category          | category               | 100 missing |
| 12 | insurance               | 4497           | category          | category               | 1292 missing |
| 13 | income                  | 4636           | category          | category               | 1153 missing |
| 14 | smoking                 | 5753           | category          | category               | 36 missing |
| 15 | alcohol                 | 5789           | float64           | float64                | 0 missing, fine |
| 16 | cocaine                 | 1430           | float64           | float64                | 4359 missing |
| 17 | contacts_count          | 5747           | Int64             | Int64 (nullable)       | 42 missing |
| 18 | public_transport_count  | 4572           | Int64             | Int64 (nullable)       | 1217 missing |
| 19 | working                 | 5761           | category          | category               | 28 missing |
| 20 | worried                 | 4518           | float64           | float64                | 1271 missing |
| 21 | covid19_positive        | 5789           | boolean           | boolean                | 0 missing |
| 22 | covid19_symptoms        | 5789           | boolean           | boolean                | 0 missing |
| 23 | covid19_contact         | 5789           | boolean           | boolean                | 0 missing |
| 24 | asthma                  | 5789           | boolean           | boolean                | 0 missing |
| 25 | kidney_disease          | 5789           | boolean           | boolean                | 0 missing |
| 26 | liver_disease           | 5789           | boolean           | boolean                | 0 missing |
| 27 | compromised_immune      | 5789           | boolean           | boolean                | 0 missing |
| 28 | heart_disease           | 5789           | boolean           | boolean                | 0 missing |
| 29 | lung_disease            | 5789           | boolean           | boolean                | 0 missing |
| 30 | diabetes                | 5789           | boolean           | boolean                | 0 missing |
| 31 | hiv_positive            | 5789           | boolean           | boolean                | 0 missing |
| 32 | other_chronic           | 5789           | boolean           | boolean                | 0 missing |
| 33 | nursing_home            | 5789           | boolean           | boolean                | 0 missing |
| 34 | health_worker           | 5789           | boolean           | boolean                | 0 missing |
| 35 | risk_infection          | 5789           | float64           | float64                | 0 missing |
| 36 | risk_infection_level    | 5789           | category          | category               | 0 missing |
| 37 | risk_mortality          | 5789           | float64           | float64                | 0 missing |


Let's convert the data types explicitly, we could use .asType() to do this in one snippet, but it might fail if there's missing or malformed data. Better to do each type explicitly

In [ ]:
# 1. Boolean columns

bool_cols = [
    'covid19_positive', 'covid19_symptoms', 'covid19_contact',
    'asthma', 'kidney_disease', 'liver_disease', 'compromised_immune',
    'heart_disease', 'lung_disease', 'diabetes', 'hiv_positive',
    'other_chronic', 'nursing_home', 'health_worker'
]
df[bool_cols] = df[bool_cols].astype('boolean')

# 2. Categorical columns
safe_cat_cols = [
    'region', 'country', 'gender', 'smoking', 'working', 'risk_infection_level'
]
df[safe_cat_cols] = df[safe_cat_cols].astype('category')

# 3. Float columns that are already clean
numeric_cols = [
    'ip_latitude', 'ip_longitude', 'height', 'weight', 'bmi',
    'alcohol', 'risk_infection', 'risk_mortality'
]
df[numeric_cols] = df[numeric_cols].astype('float64')

# --------------------------
# 4. Columns to preprocess later (commented)
# --------------------------
# survey_date -> convert to datetime after fixing bad/missing values
# df['survey_date'] = pd.to_datetime(df['survey_date'], errors='coerce')

# age -> ordered category
# age_order = ['0_10','10_20','20_30','30_40','40_50','50_60','60_70','70_80','80_90','90_100']
# df['age'] = pd.Categorical(df['age'], categories=age_order, ordered=True)

# blood_type, insurance, income, cocaine, contacts_count, public_transport_count, worried
# -> preprocess missing / malformed values before conversion

Double check

In [ ]:
df.info()

## Task 2 - Data Exploration

Task 2.1 - Identify missing values, outliers, inappropriate or inconsistent values, or errors in the
data. List the variables with the identified problems.

Task 2.2 - Identify and report the skewness present in the variables.  

In [ ]:
# Numeric columns (ready to calculate skew)
numeric_columns_df = df.select_dtypes(np.number).columns

In [ ]:
for col in numeric_columns_df:
    print(f"Skewness of {col}:", df[col].skew())
    plt.figure(figsize=(10,4))
    sns.histplot(df[col])
    plt.title(col)
    plt.tight_layout()
    plt.show()


Task 2.3 - If you need to change the type of some variables in order to identify the problems of
these variables, e.g., check the skewness of some categorical variables, you need to
report the changes.  

In [ ]:
# Need to figure out here which categorical variables we need to calculate skew on

Task 2.4 - Describe how you have identified these problems with evidence for each of the
identified problems, e.g., plots, your Python code snippets, and outputs produced by
your code.

In [ ]:
# Need to figure out 2.3 first

Task 2.5 - Answer the following questions

(a). What is the average contacts_count of a patient from the US who is COVID-19
positive?  

In [ ]:
# Filter only covid positive people
positive_cases = df[df['covid19_positive'] == 1]

# Calculate average contacts
average_contacts = positive_cases['contacts_count'].mean()

print("Average contacts for COVID-19 positive people:", average_contacts)

(b). Which country has the highest number of covid19_positive?

In [ ]:
# Filter only covid positive people
positive_cases = df[df['covid19_positive'] == 1]

# Calculate value counts for each country
country_counts = positive_cases['country'].value_counts()

# Calculate top country
top_country = country_counts.idxmax()

print("Country with the highest number of covid19_positive:", top_country)

(c). How many health workers in the dataset? and how many of them are covid19 positive?

In [ ]:
# Filter only covid positive people
health_workers = df[df['health_worker'] == 1]

# Calculate covid positive counts
covid_positive_workers = health_workers[health_workers['covid19_positive'] == 1]

print("Number of health workers in this dataset:", len(health_workers))
print("Number of Covid positive health workers:", len(covid_positive_workers))

(d). Which are the top-five countries according to the average mortality risk?

In [ ]:
# Calculate value counts for each country
avg_mortality_risk_by_country = df.groupby('country')['risk_mortality'].mean().reset_index()

# Sort descending and show just the top five countries
avg_mortality_risk_by_country.sort_values(by='risk_mortality', ascending=False).head(5)

## Task 3 - Data Preparation

Summarize your findings from your data exploration task.

Process the dataset to fix the identified data quality problems.

Elaborate on the data preparation steps, e.g., data cleaning, transformation, or
normalization.

Demonstrate the data preparation by including screenshot(s) of your Python code
snippets and its outputs that show how you corrected the identified data quality
problems in this dataset.

Describe here if you create any new derived variables and describe why you need these
derived variables.  

## Task 4 - Variable Correlation Analysis

Examine the correlations of the variables pairwisely, including both numerical and
categorical variables.

In [ ]:
numeric_columns_df = df.select_dtypes(np.number)

In [ ]:
plt.figure(figsize=(12,8))
sns.heatmap(numeric_columns_df.corr(),annot=True,cmap='viridis',vmin=-1,vmax=1)

Identify the highly correlated variable pairs and elaborate on how to treat these
variables in predictive machine learning models, especially the variables
covid19_positive and risk_infection, how would you handle these two
variables in predictive machine learning models based on your findings?  

Detail how you have identified the relationships by using suitable functions and
visualisation plots.